# Week 6 (Exercises)

## Exercise 6 & 30 from chapter 3

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm

### Exercise 6
* Describe the class of strings matched by the following regular expressions.

a) [a-zA-Z]+
<br>
b) [A-Z][a-z]*
<br>
c) p[aeiou]{,2}t
<br>
d) \d+(\.\d+)?
<br>
e) ([^aeiou][aeiou][^aeiou])*
<br>
f) \w+|[^\w\s]+

Test your answers using nltk.re_show().

A) The string that mathes this particular regex patterns are strings containing lower case and upper case letters with an arbitary length due to the plus operator which states one or more previous

In [2]:
word = 'aaaAAA'
re.findall(r'[a-zA-Z]+', word)

['aaaAAA']

In [3]:
word = 'AAAAaaaaA'
re.findall(r'[A-Z][a-z]*', word)

['A', 'A', 'A', 'Aaaaa', 'A']

B) The above regex pattern mathes strings that has capital letters and then it also matches capital letters followed by a lower case letter, then the Kleene Closure operator ensures that we also get mathces where an uppercase letter is followed by multiple lower case letters.

word = 'Hej med dig'
re.findall(r'[A-Z][a-z]*', word)

Finds words that starts with an upper case or just upper case letters.

C) Finds words that starts with a p and is then followed by two or less vowels from the set {aeiou} and ends with an t.

In [4]:
word = 'paat'
re.findall(r'p[aeiou]{,2}t', word)

['paat']

D) It provides the fraction of real numbers

In [5]:
word = '100.001'
re.findall(r'\d+(.\d+)?', word)

['.001']

E) The string that mathces this regex search is a consonant followed by a vowel and then a consonant, then the Kleene closure operator is used saying zero or more of previous, which in this case is the sequence stated.

In [6]:
word = 'kak'
re.findall(r'([^aeiou][aeiou][^aeiou])*', word ) 

['kak', '']

F) It finds all possible character strings or number strings, could be promising for tokenization purposes, given that it would seperate words and numbers into seperate tokens.

In [7]:
word = 'Jeg boede på lindegårdsvej nr 16' 
tokens = re.findall(r'\w+|[^\w\s]+', word ) 

## Exercise 30
◑ Use the Porter Stemmer to normalize some tokenized text, calling the stemmer on each word. Do the same thing with the Lancaster Stemmer and see if you observe any differences

In [8]:
from nltk import word_tokenize
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
... is no basis for a system of government.  Supreme executive power derives from
... a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = word_tokenize(raw)

porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
print([porter.stem(t) for t in tokens])
print('\n')
print([lancaster.stem(t) for t in tokens])

['denni', ':', 'listen', ',', 'strang', 'women', 'lie', 'in', 'pond', 'distribut', 'sword', '...', 'is', 'no', 'basi', 'for', 'a', 'system', 'of', 'govern', '.', 'suprem', 'execut', 'power', 'deriv', 'from', '...', 'a', 'mandat', 'from', 'the', 'mass', ',', 'not', 'from', 'some', 'farcic', 'aquat', 'ceremoni', '.']


['den', ':', 'list', ',', 'strange', 'wom', 'lying', 'in', 'pond', 'distribut', 'sword', '...', 'is', 'no', 'bas', 'for', 'a', 'system', 'of', 'govern', '.', 'suprem', 'execut', 'pow', 'der', 'from', '...', 'a', 'mand', 'from', 'the', 'mass', ',', 'not', 'from', 'som', 'farc', 'aqu', 'ceremony', '.']


The most noticable difference is that the porter stemmer manage to change lying to lie instead of just removing the end. The lanchester stemmer also stems listen to list, which essentially changes the meaning of the word entirely. 

# Prelude to part 2 - Data

Importing reddit submissions from wallstreetbets posted within January 1st and December 31st 2020 with content in English. 

In [9]:
submissions = pd.read_csv("wallstreet_subs.csv", sep=',')
submissions

FileNotFoundError: [Errno 2] No such file or directory: 'wallstreet_subs.csv'

# Part 2 - Words that characterize stocks discussed on r/wallstreetbets

Exercise 2: Most discussed stocks in r/wallstreebets. GME is only one among many stocks people have discussed in r/wallstreetbets. In this exercise, we will find the most discussed stocks on wallstreetbets. Stocks are identified by their Ticker symbol. A Ticker symbol is nothing but a string consisting of letters and numbers, and is typically quite short. For example the Gamestop symbol is GME, Amazon is AMZN, Alphabet is GOOGL...

1) Regex pattern to find words starting with a $

In [ ]:
word = 'I fucking love this $GME stuff'
re.findall(r'\$\w+', word)

In [ ]:
word = 'I fucking love this $AMZN stuff'
re.findall(r'\$[a-zA-Z]+', word)

2) Load the wallstreetbets submission dataset as a Pandas DataFrame and create a new column containing both the title and the textual content of each submission (as one long string). We refer to this as the text of the submission.

In [ ]:
submissions['text'] = submissions['title'] + ' ' + submissions['selftext']
submissions

3) For each submission, find all ticker symbols (those preceded by a dollar sign) contained in the text. Use the function re.findall, and the regular expression you created in point 1). Some tips for success:
* Remove matches that are definetly not stock symbols
* Convert all matches to uppercase
* Remove the dollar sign at the beginning of the symbol (e.g. $gme → GME).

In [ ]:
def get_ticker(word: str)->list:
    return re.findall(r'\$[a-zA-Z]{1,5}', word)

#Setting charachter length constraint due to the following: https://www.investopedia.com/terms/s/stocksymbol.asp

In [ ]:
submissions['ticker'] = submissions["text"].map(get_ticker)

In [ ]:
print(f'Initial number of submissions using ticker before filtering: {len(submissions[submissions.ticker.astype(bool)])}')

In [ ]:
for i in tqdm(range(len(submissions))):
    ticker_list = submissions.iloc[i, :]['ticker']
    temp = []
    for j in ticker_list:
        temp.append(re.sub(r'\$', '', j).upper())
    submissions['ticker'][i] = temp

In [ ]:
submissions.iloc[100,:]

In [ ]:
#Trying to validate the approach by looking at the unique tickers
(list(set([a for b in submissions.ticker.tolist() for a in b])))

By a quick inspection of the above results, it becomes evident that our current approach finds all words which start with an $, which also assemble common words. Examples which can be found in the above is words such as POPCORN, SUPERBOWL etc.

4) Create a list containing the top 15 Ticker Symbols by number of occurrences. GME should be among them. If it is not, check again your analysis and/or come talk to me. Google the top 15 symbols and find the corresponding company names. Are they known companies or not?

In [ ]:
fdist2 = nltk.FreqDist((list([a for b in submissions.ticker.tolist() for a in b])))
fdist2.most_common(15)

In [ ]:
fdist2.plot(15, cumulative=True)

## Exercise 3: TF-IDF and the stocks discussed on r/wallstreetbets. 
The goal for this exercise is to find the words charachterizing each of the stocks discussed on r/wallstreetbets. We will focus on the top 15 stocks we idenfied in Exercise 2, and we will of course use TF-IDF.

First, check out the wikipedia page for TF-IDF. Explain in your own words the point of TF-IDF.
* What does TF stand for?
TF stands for term-frequency and essentially how frequent a token occurs in a document.
<br>
* What does IDF stand for?
<br>
IDF stands for inverse document frequence, which is the inverse of how often a token occurs in the corpora. 


Tokenize the text of each submission. Create a column tokens in your dataframe containing the tokens. Remember the bullets below for success.
* If you dont' know what tokenization means, go back and read Chapter 3 again. The advice to go back and check Chapter 3 is valid for every cleaning step below.
* Exclude punctuation.
* Exclude URLs
* Exclude stop words (if you don't know what stop words are, go back and read NLPP1e again).
* Exclude numbers (since they're difficult to interpret in the word cloud).
* Set everything to lower case.
* Note that none of the above has to be perfect. And there's some room for improvisation. You can try using stemming. In my own first run the results didn't look so nice, because some submissions repeat certain words again and again and again, whereas other are very short. For that reason, I decided to use the unique set of words from each submission rather than each word in proportion to how it's actually used. Choices like that are up to you.

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    #Removing urls
    text = re.sub(r'http\S+', '', text)
    #Tokenize using nltk
    word_tokens = word_tokenize(text.lower())
    #removing none letter characters and stop words
    filtered_sentence = [w for w in word_tokens if w not in stop_words and w.isalpha()]
    #Conduct stemming
    processed_text = [porter.stem(t) for t in filtered_sentence]
    return processed_text
    
submissions['tokens'] = submissions['text'].progress_apply(lambda x: preprocess(x))


3) Find submissions discussing at least one of the top 15 stocks you identified above. To do so:
* Create a function that finds the intersection between a list of tokens and your list of top 15 stocks. For example, your function applied to the tokens: "[Here, TSLA, submission, GME]" should return ["TSLA","GME"]. (Optional: you can also try to included cases in which the list of tokens contains a company name among your top 15. For example the function applied to "[Here, Gamestop, submission]" could return ['GME'].)
* Create a new column stock in your DataFrame, containing the output of your function applied to the text column. Values in this column should be lists.
* Handle cases where one post discusses more than one stock by applying the function explode to the stock column. This will duplicate submissions associated to multiple stocks. After exploding, the values included in the stock column should be strings.
* Handle cases where none of the selected stocks is discussed by replacing Nan values, for example with "Other".

In [ ]:
top15_map = {
 'tesla': 'TSLA',
 'virgingalactic': 'SPCE',
 'palantir': 'PLTR',
 'microsoft': 'MSFT',
 'tesla':'TSLA',
 'amazon':'AMZN',
 'zoom':'ZM',
 'alibaba':'BABA',
 'gamestop':'GME',
 'disney':'DIS',
 'boeing':'BA'   
}

fdist2 = nltk.FreqDist((list([a for b in submissions.ticker.tolist() for a in b])))
top15 = [porter.stem(x[0]).lower() for x in fdist2.most_common(15)]
print(top15)
def inter_top_stock(tokens):
    return list(set(tokens) & set(top15))

submissions['stock'] = submissions['tokens'].apply(lambda x: inter_top_stock(x))


In [ ]:
submissions[submissions.stock.astype(bool)]

In [ ]:
#Exploding stock column
submissions = submissions.explode('stock')
submissions['stock'] = submissions['stock'].fillna('other')
submissions

4) Now, we want to find out which words are important for each stock, so we're going to create several large documents, one for each stock. Each document includes all the tokens related to the same stock. We will also have a document including discussions that do not relate to the top 15 stocks.

In [ ]:
corpora = {}
for i in top15:
    temp = submissions[submissions['stock']==i]
    corpora[i] = [item for sublist in temp['tokens'].tolist() for item in sublist]

5) Now, we're ready to calculate the TF for each word. Use the method of your choice to find the top 5 terms within 5 stocks of your choice.

   * Describe similarities and differences between the stocks.
   * Why aren't the TFs not necessarily a good description of the stocks?
   * Next, we calculate IDF for every word.
   * What base logarithm did you use? Is that important?


In [ ]:
#We could use sklearn countvectorizer but lets try to use nltk
#The 5 chosen are 'msft' 'gme' 'dis''aapl', 'amzn'
stock5 = ['msft','gme','tsla','aapl','amzn']
for i in stock5:
    fdist = nltk.FreqDist(corpora[i])
    print(f'The 5 most frequent words associated with {i.upper()} stock')
    print(fdist.most_common(5))
    print('\n')

The similarities between common words among the five stocks is distictive for instance is the 'stock' among all of the most common words and the same goes for the stock's name itself. Furthermore, it also noticable that all mentions words which is related to earning money in some sense. This demonstrates one of the shortcomings of TF given that it only counts the number of occurrences, and we might have taken stop words into account by removing them, but at the current moment we have not accomodated for domain specific common words such as stock. This is where TF-IDF becomes relevant because the IDF exactly attempts to weight with the inverse of the document frequency meaning that less emphasis should be attributed to domain common words.

$$ IDF(t) = log(\frac{Total\ number\ of\ documents}{Number\ of\ documents\ with\ term\ t})$$

In [ ]:
N = 5
temp = []
for i in stock5:
    temp.append(corpora[i])
terms = set([item for sublist in temp for item in sublist])

idf_dict = {}
for term in tqdm(terms):
    counter = 0
    for i in stock5:
        if term in set(corpora[i]):
            counter += 1
    idf_dict[term] = np.log((N)/(counter))

Apparently then the log base is arbitrary given that it does not matter because of the following: https://stackoverflow.com/questions/56002611/when-to-use-which-base-of-log-for-tf-idf


6) We're ready to calculate TF-IDF. Do that for the 5 stock of your choice.

    * List the 10 top TF words for each stock.
    * List the 10 top TF-IDF words for each stock.
    * Are these 10 words more descriptive of the stock? If yes, what is it about IDF that makes the words more informative

In [ ]:
for i in stock5:
    print(f'###########\n## {i} ##\n##########')
    fdist = nltk.FreqDist(corpora[i])
    print(f'The 10 most frequent words associated with {i.upper()} stock:')
    print(fdist.most_common(10))
    fdist = dict(fdist)
    print(f'The top 10 TF-idf words associated with {i.upper()} stock:')
    tf_idf_dict = {}
    for key in fdist:
        tf_idf_dict[key] = idf_dict[key] * fdist[key]
    print(sorted(tf_idf_dict.items(), key=lambda x: x[1], reverse=True)[:10])
    print('\n')

Exercise 4: The Wordcloud. It's time to visualize our results!

* Install the WordCloud module.
* Now, create word-cloud for each stock. Feel free to make it as fancy or non-fancy as you like.
* Comment on the results. Are these words to be expected? Is there anything that is surprising?



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
#Using word frequencies
for i in stock5:
    wordcloud = WordCloud(background_color='white').generate_from_text(' '.join(corpora[i]))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(i)
    plt.show()

In [ ]:
#Using word tf-idf weighting
for i in stock5:
    fdist = dict(nltk.FreqDist(corpora[i]))
    tf_idf_dict = {}
    for key in fdist:
        tf_idf_dict[key] = idf_dict[key] * fdist[key]
    wordcloud = WordCloud(background_color='white').generate_from_frequencies(tf_idf_dict)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(i)
    plt.show()

In [ ]:
submissions[submissions['text'].str.lower().str.contains("powerfleet")]